In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [2]:
import sys

In [3]:
project_path = '/content/drive/My Drive/ML project 2/'
module_path = "/content/drive/My Drive/ML project 2/levin"
sys.path.append(module_path)
sys.path.append(project_path)

In [4]:
!pip install -U segmentation-models-pytorch

In [5]:
import os
import torchvision.transforms as T
import torch
import json
from levin.train_evaluate import TrainAndEvaluate
from levin.models.unetpp.unetpp_dataset_submission import UnetPlusPlusRoadSegmentationDatasetSubmission
from levin.models.unetpp.unetpp import UnetPlusPlus
from levin.loss_functions import ComboLoss, DiceLoss, TverskyLoss, WeightedBCEWithLogitsLoss
from torchvision.transforms.functional import InterpolationMode
from sklearn.model_selection import train_test_split
from torch.utils.data import DataLoader
from levin import helpers
from levin.create_submission import CreateSubmission
from levin.helpers import get_test_images

In [6]:
encoder_name="resnet50"
encoder_weight="imagenet"

In [7]:
image_dir = os.path.join(project_path, "data", "test_set_images")

In [8]:
model_path = os.path.join(module_path, "trained_models", "unet++", "ComboLoss", "trained_model_epoch9.pt")

In [9]:
test_images = get_test_images(image_dir)
test_images.sort(key=lambda x: int(os.path.basename(x).split('_')[1].split('.')[0]))
test_images

['/content/drive/My Drive/ML project 2/data/test_set_images/test_1/test_1.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_2/test_2.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_3/test_3.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_4/test_4.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_5/test_5.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_6/test_6.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_7/test_7.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_8/test_8.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_9/test_9.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_10/test_10.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_11/test_11.png',
 '/content/drive/My Drive/ML project 2/data/test_set_images/test_12/test_12.png',
 '/content/drive/My Drive/ML proje

In [10]:
test_dataset = UnetPlusPlusRoadSegmentationDatasetSubmission(image_dir, test_images, encoder_name, encoder_weight)
test_dataloader = DataLoader(test_dataset, batch_size=1, shuffle=False)

In [11]:
model = UnetPlusPlus(encoder_name=encoder_name, encoder_weight=encoder_weight, in_channels=3, num_labels=1)
model.model.load_state_dict(torch.load(model_path))

<ipython-input-11-1b64d787c653>:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  model.model.load_state_dict(torch.load(model_path))


<All keys matched successfully>

In [12]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device {device}")

Using device cuda


In [13]:
submission_path = os.path.join(module_path, "trained_models", "unet++", "ComboLoss", "submission")
create_sub = CreateSubmission(model, test_dataloader, 0.25, submission_path, device)

In [14]:
create_sub.create_and_save_predictions()

Saving prediction:  1
Saving prediction:  2
Saving prediction:  3
Saving prediction:  4
Saving prediction:  5
Saving prediction:  6
Saving prediction:  7
Saving prediction:  8
Saving prediction:  9
Saving prediction:  10
Saving prediction:  11
Saving prediction:  12
Saving prediction:  13
Saving prediction:  14
Saving prediction:  15
Saving prediction:  16
Saving prediction:  17
Saving prediction:  18
Saving prediction:  19
Saving prediction:  20
Saving prediction:  21
Saving prediction:  22
Saving prediction:  23
Saving prediction:  24
Saving prediction:  25
Saving prediction:  26
Saving prediction:  27
Saving prediction:  28
Saving prediction:  29
Saving prediction:  30
Saving prediction:  31
Saving prediction:  32
Saving prediction:  33
Saving prediction:  34
Saving prediction:  35
Saving prediction:  36
Saving prediction:  37
Saving prediction:  38
Saving prediction:  39
Saving prediction:  40
Saving prediction:  41
Saving prediction:  42
Saving prediction:  43
Saving prediction:  

In [15]:
mask_path = os.path.join(submission_path, "prediction", "predictions")
mask_filenames = os.listdir(mask_path)
mask_filenames = [os.path.join(mask_path, f) for f in mask_filenames]
mask_filenames

['/content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions/prediction_1.png',
 '/content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions/prediction_2.png',
 '/content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions/prediction_3.png',
 '/content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions/prediction_4.png',
 '/content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions/prediction_5.png',
 '/content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions/prediction_6.png',
 '/content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions/prediction_7.png',
 '/content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/pr

In [16]:
submission_name = os.path.join(submission_path, "submission.csv")
create_sub.masks_to_submission(submission_name, *mask_filenames)

Filename: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions/prediction_1.png, number: 1
Filename: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions/prediction_2.png, number: 2
Filename: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions/prediction_3.png, number: 3
Filename: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions/prediction_4.png, number: 4
Filename: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions/prediction_5.png, number: 5
Filename: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions/prediction_6.png, number: 6
Filename: /content/drive/My Drive/ML project 2/levin/trained_models/unet++/ComboLoss/submission/prediction/predictions